In [7]:

#这个是对的
import os
import nltk
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import random
from nltk.corpus import stopwords
import operator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

from sklearn.feature_extraction.text import CountVectorizer


path = os.getcwd() + "\\bbc"
list_catagory_files = os.listdir(path)
print(list_catagory_files)
num_catagory = len(list_catagory_files)



def text_label_list(cata_path):
    label = 0
    all_text_list = []
    all_label_list = []
    catagory_file_num = []
    
    for catagory in list_catagory_files:
        catagory_file = path + "\\" + catagory
        print(catagory_file)
        if catagory_file.endswith('.TXT'):
            continue
        else:
        
            file_num = len(os.listdir(catagory_file))
            catagory_file_num.append(file_num)
            print(file_num)
            print(catagory)
            for i in range(1,file_num + 1):
                all_label_list.append(label)
                j = "%03d" % i
                with open("\\".join([catagory_file, str(j) + ".txt"])) as fp:
                    lines =  fp.readlines()
                    article = []    
                    for line in lines:
                        article.append(line.strip('\n'))
                        
                    all_text_list.append(article)
        label += 1
    return all_text_list, all_label_list, catagory_file_num

all_text_list_, all_label_list_, catagory_file_num = text_label_list(list_catagory_files)



        
# get full dataset in order to split our dataset into training and test
dataset_full=[] 
for text, label in zip(all_text_list_, all_label_list_):
    dataset_full.append((text,label))
        

def get_train_test_split(dataset_full,ratio):
    pre_train_set=[]
    pre_test_set=[]
    
    size_dataset_full = len(dataset_full)
    test_size = int(round(size_dataset_full * ratio))
    list_test_indices=random.sample(range(size_dataset_full), test_size)
    
    for i,text in enumerate(dataset_full):
        if i in list_test_indices:
            pre_test_set.append(text)
        else:pre_train_set.append(text)
        
    return pre_train_set,pre_test_set
# get training set, test set
train_set, test_set = get_train_test_split(dataset_full,0.2)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(test_set)))


# get training set, test set, development set
new_test_set, new_dev_set = get_train_test_split(test_set,0.5)

new_train_set=train_set
random.shuffle(new_train_set)
random.shuffle(new_dev_set)
random.shuffle(new_test_set)
print ("Size training set: "+str(len(new_train_set)))
print ("Size test set: "+str(len(new_test_set)))
print ("Size development set: "+str(len(new_dev_set)))





def get_bigram_vocab(training_set):
    one_list = []
    article_tokens = []
    for instance in training_set:
        article = ','.join(instance[0])
        article_tokens.append(article)
    
    paragraph = ",".join(article_tokens)
    one_list.append(paragraph)
    bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
    text_all_vec = bigram_vectorizer.fit_transform(one_list)
    
    return bigram_vectorizer

def get_bigram_vec(training_set, vectorizer):
    vec = vectorizer.transform(training_set).toarray()
    return vec
        
def chi_square_select(X_train,Y_train,num_features):
    fs_sentanalysis = SelectKBest(chi2, k=num_features).fit(X_train, Y_train)
    X_train_new = fs_sentanalysis.transform(X_train)
    return fs_sentanalysis, X_train_new


def mutual_info_select(X_train,Y_train,num_features):
    mfs_sentanalysis = SelectKBest(mutual_info_classif, k=num_features).fit(X_train, Y_train)
    X_train_new = mfs_sentanalysis.transform(X_train)
    return mfs_sentanalysis, X_train_new
    


def train_svm_classifier(training_set, bigram_vectorizer, chi_num, mutual_num): # Function for training our svm classifier
    X_train=[]
    Y_train=[]
    for instance in training_set:
        vector_instance_bigram=get_bigram_vec(instance[0],bigram_vectorizer)
        X_train.append(vector_instance_bigram)
        Y_train.append(instance[1])
        
    fs_sentanalysis, X_train_chi_vector = chi_square_select(X_train,Y_train,chi_num)
    mfs_sentanalysis, X_train_mut_vector = mutual_info_select(X_train_chi_vector,Y_train,mutual_num)
    
  # Finally, we train the SVM classifier 
    svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
    svm_clf.fit(X_train_mut_vector,Y_train)
    return svm_clf, fs_sentanalysis, mfs_sentanalysis


# print(all_text_list[-1:])
print(all_text_list_[1313])
print(all_label_list_[1313])


# def train_svm_classifier_test(training_set,bigram_vectorizer): # Function for training our svm classifier
#     X_train=[]
#     Y_train=[]
#     for instance in training_set:
        
#         vector_instance_bigram=get_bigram_vec(instance[0],bigram_vectorizer)
#         X_train.append(vector_instance_bigram)
#         Y_train.append(instance[1])
        
    
#   # Finally, we train the SVM classifier 
#     svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
#     svm_clf.fit(X_train,Y_train)
#     return svm_clf

print(dataset_full[0])

bigram_vec = get_bigram_vocab(dataset_full)


svm_clf, fs_sentanalysis, mfs_sentanalysis = train_svm_classifier(new_train_set, bigram_vec,500, 250)




X_test=[]
Y_test=[]
for instance in new_test_set:   #注意是test_set,上面的描述也错了
    vector_instance=get_vector_text(vocabulary,instance[0])
    X_test.append(vector_instance)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
Y_test_gold=np.asarray(Y_test)


from sklearn.metrics import classification_report

Y_text_predictions = svm_clf.predict(mfs_sentanalysis.transform(fs_sentanalysis.transform(X_test)))
print(Y_text_predictions)
print(classification_report(Y_test_gold, Y_text_predictions))


['business', 'entertainment', 'politics', 'README.TXT', 'sport', 'tech']
C:\Users\c2008016\Desktop\Programs\coursework\bbc\business
510
business
C:\Users\c2008016\Desktop\Programs\coursework\bbc\entertainment
386
entertainment
C:\Users\c2008016\Desktop\Programs\coursework\bbc\politics
417
politics
C:\Users\c2008016\Desktop\Programs\coursework\bbc\README.TXT
C:\Users\c2008016\Desktop\Programs\coursework\bbc\sport
511
sport
C:\Users\c2008016\Desktop\Programs\coursework\bbc\tech
401
tech
Size training set: 1780
Size test set: 445
Size training set: 1780
Size test set: 223
Size development set: 222
['Claxton hunting first major medal', '', "British hurdler Sarah Claxton is confident she can win her first major medal at next month's European Indoor Championships in Madrid.", '', 'The 25-year-old has already smashed the British record over 60m hurdles twice this season, setting a new mark of 7.96 seconds to win the AAAs title. "I am quite confident," said Claxton. "But I take each race as it

MemoryError: 

In [45]:
# get full dataset in order to split our dataset into training and test
dataset_full=[] 
for text, lable in zip(all_text_list, all_lable_list):
    dataset_full.append((text,lable))

# print(dataset_full[0])
print(dataset_full[509])


(['Lacroix label bought by US firm\n', '\n', 'Luxury goods group LVMH has sold its loss-making Christian Lacroix clothing label to a US investment group.\n', '\n', "The Paris-based firm has been shedding non-core businesses and focusing on its most profitable brands including Moet & Chandon champagne and Louis Vuitton. LVMH said the French designer's haute couture and ready-to-wear labels had been purchased by the Falic Group for an unspecified sum. The Falic Group bought two cosmetics labels from LVMH in 2003. The sale of the Lacroix label comes as many fashion houses are struggling to make money from their expensive haute couture ranges. The Florida-based Falic group, which also runs a chain of 90 duty free stores in the US, said it planned to expand the brand by opening new stores. Mr Lacroix said he planned to stay at the label he founded in 1987 although exact details are still to be confirmed.\n"], 0)


In [51]:
def get_train_test_split(dataset_full,ratio):
    pre_train_set=[]
    pre_test_set=[]
    
    size_dataset_full = len(dataset_full)
    test_size = int(round(size_dataset_full * ratio))
    list_test_indices=random.sample(range(size_dataset_full), test_size)
    
    for i,text in enumerate(dataset_full):
        if i in list_test_indices:
            pre_test_set.append(text)
        else:pre_train_set.append(text)
        
    return pre_train_set,pre_test_set
# get training set, test set
train_set, test_set = get_train_test_split(dataset_full,0.2)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(test_set)))



Size training set: 1780
Size test set: 445


In [55]:
# get training set, test set, development set
new_test_set, new_dev_set = get_train_test_split(test_set,0.5)

new_train_set=train_set
random.shuffle(new_train_set)
random.shuffle(new_dev_set)
random.shuffle(new_test_set)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(new_test_set)))
print ("Size development set: "+str(len(new_dev_set)))

Size training set: 1780
Size test set: 223
Size development set: 222


In [63]:
for example in new_train_set[:2]:
    print(example)
    print("----------------------------------------------")

(["Manufacturing recovery 'slowing'\n", '\n', 'UK manufacturing grew at its slowest pace in one-and-a-half years in January, according to a survey.\n', '\n', 'The Chartered Institute of Purchasing and Supply (CIPS) said its purchasing manager index (PMI) fell to 51.8 from a revised 53.3 in December. But, despite missing forecasts of 53.7, the PMI number remained above 50 - indicating expansion in the sector. The CIPS said that the strong pound had dented exports while rising oil and metals prices had kept costs high.\n', '\n', 'The survey added that rising input prices and cooling demand had deterred factory managers from hiring new workers in an effort to cut costs. That triggered the second successive monthly fall in the CIPS employment index to 48.3 - its lowest level since June 2003. The survey is more upbeat than official figures - which suggest that manufacturing is in recession - but analysts said the survey did suggest that the manufacturing recovery was running out of steam. "

In [2]:
# list_tokens=nltk.tokenize.word_tokenize(str(all_text_list[0]))
# print(list_tokens)

lemmatizer = nltk.stem.WordNetLemmatizer()

paragraph = ",".join(all_text_list[0])
print(all_text_list[0])
all_star = [["curry"],["butler"]]
for i, word in enumerate(all_star):
    print(i)
    print(all_star[i])
    print(word)
# print(paragraph)

sentence_split=nltk.tokenize.sent_tokenize(paragraph)
print(sentence_split)
list_tokens_sentence=nltk.tokenize.word_tokenize(sentence_split[0])
print(list_tokens_sentence)
list_tokens = []
for token in list_tokens_sentence:
      list_tokens.append(lemmatizer.lemmatize(token).lower())
print(list_tokens)

['Ad sales boost Time Warner profit\n', '\n', 'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.\n', '\n', 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.\n', '\n', "Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet cust

In [17]:
lemmatizer = nltk.stem.WordNetLemmatizer()

# def get_list_tokens(text_list):
#     paragraph = ",".join(text_list)
#     sentence_split=nltk.tokenize.sent_tokenize(paragraph)
    
#     list_tokens=[]
#     for sentence in sentence_split:
#         list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
#         for token in list_tokens_sentence:
#             list_tokens.append(lemmatizer.lemmatize(token).lower())
#     return list_tokens

    


# paragraph += ",".join(all_text_list[0])
# list_0 = get_list_tokens(paragraph)
# print(list_0)
# print(len(all_text_list))

def all_tokens(text_list):
    list_all = []
    for i, word in enumerate(text_list):
#     if i == 2:
#         break
#     else:
        paragraph = ",".join(text_list[i])
        list_ = get_list_tokens(paragraph)
#         print(paragraph)
#         print(list_)
#         print("------------" + str(i) + "-------------")
        
        list_all.extend(list_)
    return list_all

list_0 = all_tokens(all_text_list)
print(list_0[0:250])

# print(paragraph)
# for i in range(len(all_text_list)+1):
#     paragraph += ",".join(all_text_list[i])

# print(list_)
# print(list_[:250])


['ad', 'sale', 'boost', 'time', 'warner', 'profit', ',', ',', 'quarterly', 'profit', 'at', 'us', 'medium', 'giant', 'timewarner', 'jumped', '76', '%', 'to', '$', '1.13bn', '(', 'â£600m', ')', 'for', 'the', 'three', 'month', 'to', 'december', ',', 'from', '$', '639m', 'year-earlier', '.', ',', ',', 'the', 'firm', ',', 'which', 'is', 'now', 'one', 'of', 'the', 'biggest', 'investor', 'in', 'google', ',', 'benefited', 'from', 'sale', 'of', 'high-speed', 'internet', 'connection', 'and', 'higher', 'advert', 'sale', '.', 'timewarner', 'said', 'fourth', 'quarter', 'sale', 'rose', '2', '%', 'to', '$', '11.1bn', 'from', '$', '10.9bn', '.', 'its', 'profit', 'were', 'buoyed', 'by', 'one-off', 'gain', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'warner', 'bros', ',', 'and', 'le', 'user', 'for', 'aol', '.', ',', ',', 'time', 'warner', 'said', 'on', 'friday', 'that', 'it', 'now', 'owns', '8', '%', 'of', 'search-engine', 'google', '.', 'but', 'it', 'own', 'internet', 'business', ',', 'aol', ',', 'h

In [68]:
from nltk.corpus import stopwords
import operator
stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['/','(',')','{','}','@','|',';','\n','#','+','_','.',',','``',"''",]
stopwords.extend(new_stopwords)



dict_word_frequency={}
for word in list_0:
    if word in stopwords: continue
    if word not in dict_word_frequency: dict_word_frequency[word]=1
    else: dict_word_frequency[word]+=1

sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)
print(len(sorted_list))
#   特征数
# print(sorted_list[:15])
# vocabulary=[]
# for word,frequency in sorted_list:
#     vocabulary.append(word)

# print(vocabulary[:10])


32571


In [ ]:
def get_vocabulary(training_set, num_features): # Function to retrieve vocabulary
    dict_word_frequency={}
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            if word in stopwords: continue
            if word not in dict_word_frequency: dict_word_frequency[word]=1
            else: dict_word_frequency[word]+=1
    sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:num_features]
    vocabulary=[]
    for word,frequency in sorted_list:
        vocabulary.append(word)
    return vocabulary   #word frequency元组

In [71]:
def get_vector_text(list_vocab,text_list):
    vector_text=np.zeros(len(list_vocab))
    paragraph = ",".join(text_list)
    list_tokens_string=get_list_tokens(paragraph)
    for i, word in enumerate(list_vocab):
        if word in list_tokens_string:
            vector_text[i]=list_tokens_string.count(word)
    return vector_text

vec = get_vector_text(vocabulary,all_text_list[0])
print(vec)
print(len(vec))

[  0.   0.   0.   0.   9.   0.   0.   0.   0.   0.   8.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   9.   5.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 

[0, 1, 4, 9, 16]
1
1


In [2]:
# lis = [["stephen curry"],["micheal jordan"],["lebron james"],["kevin durant"],["kyre irving"],["klay tompson"]]
# lis_1 = [2,4,6]

# list_0 = lis[0:lis_1[0]]
# print(list_0)
# list_1 = lis[lis_1[0]:lis_1[1]]
# print(list_1)


# seperate the all_text_list into different list according to different catagories
def sep_list(lis,cata):
    list_sep = []
    for i in range(len(cata)):
        if i == 0:
            list_sep.append(lis[i:cata[i]])
        else:
            list_sep.append(lis[cata[i-1]:cata[i]])
        
    return list_sep
    
    
list_5 = sep_list(lis,lis_1)
print(list_5)
print(len(lis_1))
print(list_5[0])
print(lis[2:4])
print(lis[4:6])

[['stephen curry'], ['micheal jordan']]
[['lebron james'], ['kevin durant']]
[[['stephen curry'], ['micheal jordan']], [['lebron james'], ['kevin durant']], [['kyre irving'], ['klay tompson']]]
3
[['stephen curry'], ['micheal jordan']]
[['lebron james'], ['kevin durant']]
[['kyre irving'], ['klay tompson']]


In [25]:


print(catagory_file_num)
new_cata = [510,896,1313,1824,2225]

print(len(new_cata))
sep_list_ = sep_list(all_text_list,new_cata)


[510, 386, 417, 511, 401]
5


In [40]:

# def train_svm_classifier(training_set, vocabulary):


# for business_text in sep_list_[0]:
#     dataset_full.append((business_text,0))


    
    
# X_train=[]
# Y_train=[]
#    
# #      这里要把所有条目都分成类得到dataset_full  然后回头从3-5开始进行数据分类（测试集，验证集等等）但这以下部分已经证实是我们训练用的格式了，有了字典集，就差训练了
# for instance in dataset_full:
#     vecto = get_vector_text(vocabulary,instance[0])  
#     X_train.append(vecto)
#     Y_train.append(instance[1])
# print(X_train[0])


2225
2225


In [64]:
def train_svm_classifier(training_set, vocabulary): # Function for training our svm classifier
    X_train=[]
    Y_train=[]
    for instance in training_set:
        vector_instance=get_vector_text(vocabulary,instance[0])
        X_train.append(vector_instance)
        Y_train.append(instance[1])
  # Finally, we train the SVM classifier 
    svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
    svm_clf.fit(np.asarray(X_train),np.asarray(Y_train))
    return svm_clf


vocabulary=get_vocabulary(new_train_set, 1000)

svm_clf=train_svm_classifier(new_train_set, vocabulary)

NameError: name 'get_vocabulary' is not defined

In [72]:


lemmatizer = nltk.stem.WordNetLemmatizer()

# paragraph = ",".join(all_text_list[0])
# print(all_text_list[0])
# print("-----------------------")
# all_star = [["curry"],["butler"]]
# for i, word in enumerate(all_star):
#     print(i)
#     print(all_star[i])
#     print(word)
# print(paragraph)

# sentence_split=nltk.tokenize.sent_tokenize(paragraph)
# print(sentence_split)
# list_tokens_sentence=nltk.tokenize.word_tokenize(sentence_split[0])
# print(list_tokens_sentence)
# list_tokens = []
# for token in list_tokens_sentence:
#       list_tokens.append(lemmatizer.lemmatize(token).lower())
# print(list_tokens)




def get_list_tokens(text_list):
    paragraph = ",".join(text_list)
    sentence_split=nltk.tokenize.sent_tokenize(paragraph)
    
    list_tokens=[]
    for sentence in sentence_split:
        list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
        for token in list_tokens_sentence:
            list_tokens.append(lemmatizer.lemmatize(token).lower())
    return list_tokens

new = get_list_tokens(all_text_list[0])
print(new)

def get_para(text_list):
    paragraph = ",".join(text_list)
    sentence_split = nltk.tokenize.sent_tokenize(paragraph)
    return sentence_split

para = get_para(all_text_list[0])
print(para)

['ad', 'sale', 'boost', 'time', 'warner', 'profit', ',', ',', 'quarterly', 'profit', 'at', 'us', 'medium', 'giant', 'timewarner', 'jumped', '76', '%', 'to', '$', '1.13bn', '(', 'â£600m', ')', 'for', 'the', 'three', 'month', 'to', 'december', ',', 'from', '$', '639m', 'year-earlier', '.', ',', ',', 'the', 'firm', ',', 'which', 'is', 'now', 'one', 'of', 'the', 'biggest', 'investor', 'in', 'google', ',', 'benefited', 'from', 'sale', 'of', 'high-speed', 'internet', 'connection', 'and', 'higher', 'advert', 'sale', '.', 'timewarner', 'said', 'fourth', 'quarter', 'sale', 'rose', '2', '%', 'to', '$', '11.1bn', 'from', '$', '10.9bn', '.', 'its', 'profit', 'were', 'buoyed', 'by', 'one-off', 'gain', 'which', 'offset', 'a', 'profit', 'dip', 'at', 'warner', 'bros', ',', 'and', 'le', 'user', 'for', 'aol', '.', ',', ',', 'time', 'warner', 'said', 'on', 'friday', 'that', 'it', 'now', 'owns', '8', '%', 'of', 'search-engine', 'google', '.', 'but', 'it', 'own', 'internet', 'business', ',', 'aol', ',', 'h

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

all_text_list[0]
print(all_text_list[0])
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
print(vectorizer)
X = vectorizer.fit_transform(all_text_list[0])
vec = vectorizer.get_feature_names_out()
print(vec[:50])
print(X)


NameError: name 'all_text_list' is not defined

In [76]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 1 0 ... 0 0 3]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 0]]


In [ ]:
corpus = [
...     'This is the first document.',
...     'This document is the second document.',
...     'And this is the third one.',
...     'Is this the first document?',
... ]





ValueError: Expected 2D array, got 1D array instead:
array=["Manufacturing recovery 'slowing'\n" '\n'
 'UK manufacturing grew at its slowest pace in one-and-a-half years in January, according to a survey.\n'
 '\n'
 'The Chartered Institute of Purchasing and Supply (CIPS) said its purchasing manager index (PMI) fell to 51.8 from a revised 53.3 in December. But, despite missing forecasts of 53.7, the PMI number remained above 50 - indicating expansion in the sector. The CIPS said that the strong pound had dented exports while rising oil and metals prices had kept costs high.\n'
 '\n'
 'The survey added that rising input prices and cooling demand had deterred factory managers from hiring new workers in an effort to cut costs. That triggered the second successive monthly fall in the CIPS employment index to 48.3 - its lowest level since June 2003. The survey is more upbeat than official figures - which suggest that manufacturing is in recession - but analysts said the survey did suggest th